In [ ]:
# Cell 1
from TikTokApi import TikTokApi
import pandas as pd
import csv
import time
from datetime import datetime


In [ ]:
# Cell 2
# Keyword dan jendela waktu
KEYWORD    = "maybank run 2024"
START_DT   = datetime(2024, 6, 1)
END_DT     = datetime(2024, 10, 31, 23, 59, 59)

# Fungsi bantu konversi UNIX timestamp ke datetime
def to_datetime(ts: int) -> datetime:
    return datetime.fromtimestamp(ts)


In [ ]:
# Cell 3
api = TikTokApi.get_instance()

# Ambil hingga N video (batasi misal 200)
videos = api.search_for_videos(KEYWORD, count=200)

# Filter berdasarkan tanggal post
filtered = []
for v in videos:
    dt = to_datetime(v.createTime)
    if START_DT <= dt <= END_DT:
        filtered.append(v)
print(f"▶️ Ditemukan {len(filtered)} video antara Juni–Okt 2024")


In [ ]:
# Cell 4
for v in filtered:
    vid = v.id
    post_dt = to_datetime(v.createTime).isoformat()
    filename = f"comments_{vid}.csv"
    print(f"\n⏳ Processing {vid} (posted {post_dt}) → {filename}")

    # Fetch semua komentar (maks 1000; sesuaikan jika perlu)
    comments = api.video(id=vid).comments(count=1000)

    # Simpan ke CSV
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=[
            "comment_id", "username", "text", "created_at"
        ])
        writer.writeheader()
        for c in comments:
            writer.writerow({
                "comment_id": c.id,
                "username":   c.author.username,
                "text":       c.text.replace("\n"," "),
                "created_at": to_datetime(c.createTime).isoformat()
            })
    print(f"✅ Saved {len(comments)} comments to {filename}")

    # jeda kecil biar sopan ke server
    time.sleep(2)
